<a href="https://colab.research.google.com/github/fettahyildizz/Deep-Learning/blob/main/Multilayer%20Perceptron/multilayer_fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt



In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train = x_train[:6000]
x_test = x_test[:6000]


4423680/4422102 [==============================] - 0s 0us/step


In [3]:
class BackPropogation:
    def __init__(self,class_number, learning_rate1, learning_rate2, learning_rate3, n_epoch, n_layer, n_neuron1, n_neuron2,
                 momentum):
        self.momentum = momentum
        self.learning_rate1 = learning_rate1
        self.learning_rate2 = learning_rate2
        self.learning_rate3 = learning_rate3
        self.accuracy = list()
        self.class_number = class_number
        self.n_epoch = n_epoch
        self.n_layer = n_layer
        self.n_neuron1 = n_neuron1
        self.n_neuron2 = n_neuron2
        self.bias = np.ones(1)

        self.activation_func = self.sigmfunc
        self.weight0 = None
        self.weight1 = None
        self.weight2 = None

    def sigmfunc(self, x):
        return 1 / (1 + np.exp(-x))

    def sigturev(self, x):
        return (1 / (1 + np.exp(-x))) * (1 - (1 / (1 + np.exp(-x))))

    def train(self, X, yd):
        
        self.weight1 = np.random.uniform(-0.15, 0.15, (
            self.n_neuron1, X[0].size + 1))  # İlk ağırlıkları -0.15,0.15 arasında rastgele seçeriz
        self.weight2 = np.random.uniform(-0.15, 0.15, (self.n_neuron2, self.n_neuron1 + 1))
        self.weight0 = np.random.uniform(-0.15, 0.15, (self.class_number, self.n_neuron2 + 1))
        self.grad0 = None
        self.grad1 = None
        self.grad2 = None
        self.old_weight1 = self.weight1
        self.old_weight2 = self.weight2
        self.old_weight0 = self.weight0
        self.older_weight1 = self.weight1
        self.older_weight2 = self.weight2
        self.older_weight0 = self.weight0

        for i in range(self.n_epoch):
            self.error_list = list()
            for idx, image in enumerate(X):
                
                self.older_weight1 = self.old_weight1
                self.older_weight2 = self.old_weight2
                self.older_weight0 = self.old_weight0
                self.old_weight1 = self.weight1
                self.old_weight2 = self.weight2
                self.old_weight0 = self.weight0

                image_flatten = image.flatten()
                image_flatten = image_flatten / 255  # Normalize
                
                self.image_x_biased = np.concatenate((image_flatten, self.bias))  # AĞIRLIĞA BIAS EKLENIR
                self.image_x = np.transpose(np.array([self.image_x_biased]))  # x inputları düzenli hale getirilir
                self.v1 = np.matmul(self.weight1, self.image_x)
                self.y1 = self.sigmfunc(self.v1)
                self.y1_x = np.vstack(
                    (self.y1, self.bias))  # ikinci katman için birinci katman çıkışlarına bias eklenir.
                self.y1_x_t = np.transpose(self.y1_x)

                self.v2 = np.matmul(self.weight2, self.y1_x)
                self.y2 = self.sigmfunc(self.v2)
                self.y2_x = np.vstack((self.y2, self.bias))
                self.y2_x_t = np.transpose(self.y2_x)

                

                self.v0 = np.matmul(self.weight0, self.y2_x)
                self.y0 = self.sigmfunc(self.v0)
                yd_temp = np.array(yd[idx])
                
                yd_temp = np.transpose([yd_temp])
                self.e = yd_temp - self.y0
                
                self.e_temp = np.array([self.e])
                self.et = np.transpose(self.e)
                self.E = (1 / 2) * np.matmul(self.et, self.e)
                
                self.error_list.append(float(self.E))

                self.grad0 = self.e * self.sigturev(self.v0)

                self.weight0_c = np.delete(self.weight0, -1, 1)  # ağırlığın son sütununu çıkarıp tranzpozesi alınır
                self.weight0_c_t = np.transpose(self.weight0_c)
                self.grad2 = np.dot(self.weight0_c_t, self.grad0) * self.sigturev(self.v2)

                self.weight2_c = np.delete(self.weight2, -1, 1)
                self.weight2_c_t = np.transpose(self.weight2_c)
                self.grad1 = np.dot(self.weight2_c_t, self.grad2) * self.sigturev(self.v1)

                self.grad0_np = np.array([self.grad0])
                self.grad0_np = np.transpose(self.grad0_np)
                self.grad1_np = np.array([self.grad1])
                self.grad1_np = np.transpose(self.grad1_np)
                self.grad2_np = np.array([self.grad2])
                self.grad2_np = np.transpose(self.grad2_np)

                self.y2_x_t = np.transpose(self.y2_x)
                self.y1_x_t = np.transpose(self.y1_x)
                self.image_x_t = np.array([self.image_x_biased])

                ##################AĞIRLIK GÜNCELLEME
                self.weight0 = self.weight0 + self.learning_rate3 * np.matmul(self.grad0,
                                                                              self.y2_x_t) + self.momentum * (
                                       self.old_weight0 - self.older_weight0)  # y2_x_t
                self.weight2 = self.weight2 + self.learning_rate2 * np.matmul(self.grad2,
                                                                              self.y1_x_t) + self.momentum * (
                                       self.old_weight2 - self.older_weight2)
                self.weight1 = self.weight1 + self.learning_rate1 * np.matmul(self.grad1,
                                                                              self.image_x_t) + self.momentum * (
                                       self.old_weight1 - self.older_weight1)

            self.accuracy.append(sum(self.error_list) / len(self.error_list))
            print("Epoch: " + str(i + 1) + "Loss: " + str(self.accuracy[i]))

    def fit(self, test, test_label):
        accuracy = 0
        for idx, tst in enumerate(test):
            image_flatten = tst.flatten()
            image_x_biased = np.concatenate((image_flatten, self.bias))
            image_x_t = np.array([image_x_biased])
            image_x = np.transpose(np.array([image_x_biased]))  # x inputs
            v1 = np.matmul(self.weight1, image_x)
            y1 = self.sigmfunc(v1)
            y1_x = np.vstack((y1, self.bias))  # ikinci katman için birinci katman çıkışlarına bias eklenir.
            y1_x_t = np.transpose(y1_x)

            v2 = np.matmul(self.weight2, y1_x)
            y2 = self.sigmfunc(v2)
            y2_x = np.vstack((y2, self.bias))
            y2_x_t = np.transpose(y2_x)

            v0 = np.matmul(self.weight0, y2_x)
            y0 = self.sigmfunc(v0)

            # print(y0)
            # if(y0==)
            index_max = np.argmax(y0)
            test_label_temp = np.argmax(test_label[idx])
            if(index_max == test_label_temp):
                accuracy += 1
        print("Evaluated accuracy: "+ str(accuracy/len(test)))

In [4]:
neuralNetwork = BackPropogation(class_number = 10, learning_rate1=0.2,learning_rate2=0.2,learning_rate3=0.2, n_epoch=25, n_layer=2, n_neuron1=60, n_neuron2=30, momentum=0.4)
neuralNetwork.train(x_train, y_train)

Epoch sayısı: 1Ortalama loss: 0.2975930246537047
Epoch sayısı: 2Ortalama loss: 0.17724396293693356
Epoch sayısı: 3Ortalama loss: 0.14764113641098156
Epoch sayısı: 4Ortalama loss: 0.13527889904852067


KeyboardInterrupt: ignored

In [ ]:
plt.im

In [ ]:
neuralNetwork.fit(x_test, y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


Evaluated accuracy: 0.7971666666666667


In [ ]:
plt.title("Ani hata değerleri - Epoch:25 -1.Katman:60, 2.Katman:30 nöron-Öğrenme katsayısı:0.2-0.2-0.2 Momentum:0.4")
plt.xlabel("Epoch sayısı")
plt.ylabel("Ani hata değeri")
plt.plot(neuralNetwork.accuracy)
plt.show()